# Build a cuisine recommender

In [1]:
import pandas as pd
df = pd.read_csv('../data/cleaned_cuisines.csv')
df.head()

,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [3]:
X = df.iloc[:, 2:]
y = df['cuisine']

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [5]:
from sklearn.svm import SVC
model = SVC(kernel= 'linear', C = 10, probability=True, random_state=0)
model.fit(X_train, y_train.values.ravel())
y_pred = model.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy is {accuracy}')
print('Classification report', confusion_matrix(y_test, y_pred))
print('Precision score', precision_score(y_test, y_pred, average=None))
print('Confusion matrix\n', confusion_matrix(y_test, y_pred))

Accuracy is 0.780650542118432
Classification report [[180   6  24  13  22]
 [  8 209   3   2  11]
 [ 30  11 182  13  13]
 [ 26   7  20 164   5]
 [ 22  17   6   4 201]]
Precision score [0.67669173 0.836      0.77446809 0.83673469 0.79761905]
Confusion matrix
 [[180   6  24  13  22]
 [  8 209   3   2  11]
 [ 30  11 182  13  13]
 [ 26   7  20 164   5]
 [ 22  17   6   4 201]]


Conversion to onnx

In [10]:
from skl2onnx.convert import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_types = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}}

onnx = convert_sklearn(model, initial_types=initial_types, options=options)
with open('./model.onnx', 'wb') as file:
    file.write(onnx.SerializeToString())